In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
IMAGE_SIZE = [224, 224]

train_path = 'Dataset/train'
valid_path = 'Dataset/test'

In [4]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
for layer in vgg.layers:
  layer.trainable = False

In [8]:
folders = glob('Dataset_Apple/train/*')


In [9]:
folders

['Dataset_Apple/train\\fresh Apples', 'Dataset_Apple/train\\rotten Apples']

In [10]:
x = Flatten()(vgg.output)

In [11]:
len(folders)

2

In [12]:
prediction = Dense(len(folders), activation='softmax')(x)

In [13]:
model = Model(inputs=vgg.input, outputs=prediction)

In [14]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [15]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
training_set = train_datagen.flow_from_directory('Dataset_Apple/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 4035 images belonging to 2 classes.


In [20]:
test_set = test_datagen.flow_from_directory('Dataset_Apple/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 996 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
127/127 [==============================] - 1749s 14s/step - loss: 0.2884 - accuracy: 0.8924 - val_loss: 0.1082 - val_accuracy: 0.9649
Epoch 2/10
127/127 [==============================] - 1385s 11s/step - loss: 0.0954 - accuracy: 0.9680 - val_loss: 0.0603 - val_accuracy: 0.9819
Epoch 3/10
127/127 [==============================] - 1897s 15s/step - loss: 0.0704 - accuracy: 0.9740 - val_loss: 0.0588 - val_accuracy: 0.9779
Epoch 4/10
 76/127 [================>.............] - ETA: 9:09 - loss: 0.0480 - accuracy: 0.9850

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

In [ ]:
plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc,"go-", label='Training Accuracy')
plt.plot(epochs_range, val_acc,"ro-", label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.savefig('AccurateValue_acc_points')

In [ ]:
plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss,"go-", label='Training Loss')
plt.plot(epochs_range, val_loss,"ro-", label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
plt.savefig('LossValue_loss_points')

In [26]:
pip install opencv-python


  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)


In [68]:
import cv2
image = cv2.imread('Dataset/single_prediction/rotten3.jpeg')
image_resized = cv2.resize(image, (224,224))
image = np.expand_dims(image_resized,axis=0)
print(image.shape)

(1, 224, 224, 3)


In [69]:
predict = model.predict(image)

In [66]:
print(predict)

[[0. 1.]]


In [67]:
if predict[0][0] > predict[0][1]:
    print("fresh")
else:
    print("rotten")

rotten
